In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Pre processing and Data Augmenataion
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.1,
    shear_range=0.1
)


#Create Traning Data
train_generator = train_datagen.flow_from_directory(
    'F:/Project/Final Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

#Create Validation Data
validation_generator = train_datagen.flow_from_directory(
    'F:/Project/Final Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 18677 images belonging to 2 classes.
Found 4668 images belonging to 2 classes.


In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False


In [3]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 2,422,210
Trainable params: 164,226
Non-tr

In [4]:
# Set up early stopping
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [5]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator,
                    callbacks=[early_stop])


Epoch 1/10
584/584 [==============================] - 562s 959ms/step - loss: 0.0951 - accuracy: 0.9640 - val_loss: 0.0311 - val_accuracy: 0.9891
Epoch 2/10
584/584 [==============================] - 558s 956ms/step - loss: 0.0555 - accuracy: 0.9792 - val_loss: 0.0337 - val_accuracy: 0.9897
Epoch 3/10
584/584 [==============================] - 556s 952ms/step - loss: 0.0437 - accuracy: 0.9837 - val_loss: 0.0218 - val_accuracy: 0.9925
Epoch 4/10
584/584 [==============================] - 556s 953ms/step - loss: 0.0414 - accuracy: 0.9844 - val_loss: 0.0329 - val_accuracy: 0.9908
Epoch 5/10
584/584 [==============================] - 554s 950ms/step - loss: 0.0342 - accuracy: 0.9875 - val_loss: 0.0263 - val_accuracy: 0.9899
Epoch 6/10
584/584 [==============================] - 557s 955ms/step - loss: 0.0323 - accuracy: 0.9884 - val_loss: 0.0390 - val_accuracy: 0.9844


In [6]:
model.save('project17_model.h5')
